# An Introduction to Analysing In the Spotlight Data Using Python

The purpose of this notebook is to help you become more familiar with the data structures used to store [LibCrowds](https://www.libcrowds.com/)' results data, as well as introduce you to a key Python librarys that can be used to request, manipulate, analyse and visualise that data.

The Python library used here is called [pandas](https://pandas.pydata.org/). It provides access to high-performance data analysis tools via an accessible Python interface. We will use the library to load all of our *In the Spotlight* results into a structure called a dataframe. A dataframe is a two-dimensional data structure, similar to a spreadsheet, that accepts many different kinds of input. As everything is stored in memory, rather than on disk, the only limitation to this type of data structure is going to be the amount of RAM installed on the computer. However, for any modern computer this is unlikely to be an issue until we reach tens of millions of results.

We begin by importing the pandas library.

In [1]:
import pandas

## The dataset

As mentioned above, our input for this notebook will be all of the results generated so far through the crowdsourcing projects presented on [*In the Spotlight*](https://www.libcrowds.com/collections/playbills). These results are stored as Web Annotations; a W3C standard used to make data more easily reusable online. The abstract from the [Web Annotation Data Model](https://www.w3.org/TR/annotation-model) is presented below:

> Annotations are typically used to convey information about a resource or associations between resources. Simple examples include a comment or tag on a single web page or image, or a blog post about a news article.
>
> The Web Annotation Data Model specification describes a structured model and format to enable annotations to be shared and reused across different hardware and software platforms. Common use cases can be modeled in a manner that is simple and convenient, while at the same time enabling more complex requirements, including linking arbitrary content to a particular data point or to segments of timed multimedia resources.
>
> The specification provides a specific JSON format for ease of creation and consumption of annotations based on the conceptual model that accommodates these use cases, and the vocabulary of terms that represents it.

By using this standardised structure for our final results we aim to make the crowdsourced data generated via the LibCrowds platform more easily reusable online, providing ways for researchers to answer specific questions via programmatic means. As well as the results data, we use Web Annotations to store additional user-generated data, such as image tags. 

All of these annotations are available via a public API that complies with a [standardised protocol](https://www.w3.org/TR/annotation-protocol/). The API can be used to gain programmatic access to our current results data. Depending on the objective, this method of consuming and analysing the data is likely to present advantages when compared with performing similar visualisations using offline datasets (i.e. those downloaded to your computer). Importantly, data requested via the API will always be up-to-date, as very soon after a task is completed the contributions will be analysed and the result made available via the API.

## The API

The LibCrowds Web Annotation API is available at the following location:

https://annotations.libcrowds.com/annotations/

Results are returned in containers called Annotation Collections. The JSON-LD (JSON Linked Data) structure used to model Annotation Collections comprises key-value pairs, where each key has some semantic meaning. To avoid overloading this notebook with new concepts (perhaps it's too late), we won't go into the semantic meaning of each key. For now, it is enough to understand that there are consistent ways to programmatically navigate these data structures and pull out the information required, regardless of where they are stored or how they were produced.

In addition to the standard protocol, the API includes an endpoint to which we can make a request for an entire collection of annotations, declared below.

In [2]:
IRI = 'https://annotations.libcrowds.com/export/playbills-results/'

Note that this dataset could be very large. If you were to enter the address above into a web-browser it is likely that it would run out of memory before it ever finished loading, it is primarily intended for used as a way of exporting the data programmatically. 

Further, having above extolled the virtues of using the API to access live data it may seem somewhat strange to proceed and download the entire dataset anyway. A more efficient way of using the API to search for specific data will be explored in other notebooks. However, as the primary purpose of this notebook is to introduce the structures used to store LibCrowds' data, for the sake of simplicity, in this notebook we will simply load the entire dataset into a dataframe and filter out the parts that we don't need.

We can load our *In the Spotlight* results data into a pandas dataframe with the following line of code.

In [3]:
df = pandas.read_json(IRI, orient='records')

## The data model

The LibCrowds data model describes how we use Web Annotations to model our results data. It might be worth opening up [the documentation](https://docs.libcrowds.com/data/model/) for reference; the key sections are also summarised below.

In this notebook we will be focussing on the transcription annotations. These are generated after at least three people have contributed to a single task where a task may be, for example, to transcribe a previously marked up region on an image. The contributions are normalised and the final result saved as an annotation if three of the normalised contributions match. If we don't find a match, the tasks are sent back into the queue.

The annotations are generated according to the structure of the example annotation presented below. Don't worry if this doesn't make any sense just yet, the relevant sections will be explored in a bit more detail later. For now, note that the annotations are stored as JSON-LD; a method of encoding Linked Data using JSON. Each key has some semantic meaning, making it easier for other hardware and software platforms to consume the data.

```json-ld
{
  "@context": "http://www.w3.org/ns/anno.jsonld",
  "id": "https://annotations.libcrowds.com/annotations/playbills-results/7640ddcd-6e48-4a9c-a360-3383032593b6",
  "type": "Annotation",
  "motivation": "describing",
  "created": "018-02-08T22:15:07.152Z",
  "generated": "018-02-08T22:15:07.152Z",
  "generator": [
    {
      "id": "https://github.com/LibCrowds/libcrowds",
      "type": "Software",
      "name": "LibCrowds",
      "homepage": "https://www.libcrowds.com"
    },
    {
      "id": "https://backend.libcrowds.com/api/result/42",
      "type": "Software"
    }
  ],
  "body": [
    {
      "type": "TextualBody",
      "purpose": "tagging",
      "value": "title"
    },
    {
      "type": "TextualBody",
      "purpose": "describing",
      "value": "King Lear",
      "format": "text/plain"
    }
  ],
  "target": {
    "source": "https://api.bl.uk/metadata/iiif/ark:/81055/vdc_100022589096.0x0002b7",
    "selector": {
      "conformsTo": "http://www.w3.org/TR/media-frags/",
      "type": "FragmentSelector",
      "value": "?xywh=7,1191,1962,359"
    }
  }
}
```

For our current purposes (analysis of the transcribed data), we only need to focus on a subset of these keys. Below, we will find out how to manipulate this data to locate only the parts of it we require.

## Processing the dataset

The dataframe we created earlier contains one row per annotation, where an annotation represents a single result. A column is created for each top level key found in the annotations. This is perhaps easier to understand by looking at a few rows of the dataframe below and comparing them to the example JSON-LD annotation presented above.

The following fuction can be used to return the first 5 rows in a dataframe.

In [4]:
df.head()

,body,created,creator,generated,generator,id,motivation,partOf,target,type
0,"[{u'type': u'TextualBody', u'purpose': u'descr...",2018-05-30T18:29:57Z,NaN,2018-06-04T09:43:56Z,"[{u'type': u'Software', u'homepage': u'https:/...",https://annotations.libcrowds.com/annotations/...,describing,https://api.bl.uk/metadata/iiif/ark:/81055/vdc...,https://api.bl.uk/metadata/iiif/ark:/81055/vdc...,Annotation
1,"{u'type': u'TextualBody', u'purpose': u'commen...",2018-05-30T18:31:22Z,"{u'type': u'Person', u'nickname': u'SharonE', ...",2018-06-04T09:43:56Z,"[{u'type': u'Software', u'homepage': u'https:/...",https://annotations.libcrowds.com/annotations/...,commenting,https://api.bl.uk/metadata/iiif/ark:/81055/vdc...,https://api.bl.uk/metadata/iiif/ark:/81055/vdc...,Annotation
2,"{u'type': u'TextualBody', u'purpose': u'commen...",2018-05-30T18:31:22Z,NaN,2018-06-04T09:43:56Z,"[{u'type': u'Software', u'homepage': u'https:/...",https://annotations.libcrowds.com/annotations/...,commenting,https://api.bl.uk/metadata/iiif/ark:/81055/vdc...,https://api.bl.uk/metadata/iiif/ark:/81055/vdc...,Annotation
3,"{u'type': u'TextualBody', u'purpose': u'commen...",2018-05-30T18:31:23Z,"{u'type': u'Person', u'nickname': u'Bar', u'id...",2018-06-04T09:43:56Z,"[{u'type': u'Software', u'homepage': u'https:/...",https://annotations.libcrowds.com/annotations/...,commenting,https://api.bl.uk/metadata/iiif/ark:/81055/vdc...,https://api.bl.uk/metadata/iiif/ark:/81055/vdc...,Annotation
4,"{u'type': u'TextualBody', u'purpose': u'commen...",2018-05-30T18:32:46Z,"{u'type': u'Person', u'nickname': u'SharonE', ...",2018-06-04T09:43:56Z,"[{u'type': u'Software', u'homepage': u'https:/...",https://annotations.libcrowds.com/annotations/...,commenting,https://api.bl.uk/metadata/iiif/ark:/81055/vdc...,https://api.bl.uk/metadata/iiif/ark:/81055/vdc...,Annotation


While these columns all serve a purpose and are essential for proper integration with other software platforms, it is likely that we will often only be interested in a few core parts of the dataset. Namely, the transcriptions provided by our volunteers, along with an indication of what they were transcribing. In Web Annotation terms, these would be referred to as the **body** and **target**, respectively. This is summarised in the following extract from the [Web Annotation Data Model](https://www.w3.org/TR/annotation-model/):

> An Annotation is a Web Resource. Typically, an Annotation has a single Body, which is a comment or other descriptive resource, and a single Target that the Body is somehow "about". The Annotation likely also has additional descriptive properties.

Among these additional descriptive properties is the **motivation**, which specifies the reason for the Annotation's creation. LibCrowds' results are generated with one of three possible motivations: tagging, describing and commenting. The code below selects the motivation column and returns the unique values it contains as an array.

In [5]:
df['motivation'].unique()

array([u'describing', u'commenting', u'tagging'], dtype=object)

For many of our analyses, we will only be interested in annotations with a particular motivation. For instance, for the remainder of this notebook we will be looking at annotations with the describing motivation. This is the type of annotation that we use to store our transcription data. The decision to use *describing* as the motivation here (rather than, say, *transcribing*) was taken as *describing* has a semantic meaning defined by the standard [Web Annotation Vocabulary](https://www.w3.org/TR/annotation-vocab/#describing):

> **2.3.5 describing:** The motivation for when the user intends to describe the Target, as opposed to (for example) a comment about it. 

It is possible to select only those annotations where the motivation is equal to describing and replace the current dataframe with the result, using the code below.

In [6]:
df = df[df['motivation'] == 'describing']

To verify that this has worked we can again check the unique motivations in the dataframe.

In [7]:
df['motivation'].unique()

array([u'describing'], dtype=object)

 We can also have another look at the first few rows.

In [8]:
 df.head()

,body,created,creator,generated,generator,id,motivation,partOf,target,type
0,"[{u'type': u'TextualBody', u'purpose': u'descr...",2018-05-30T18:29:57Z,NaN,2018-06-04T09:43:56Z,"[{u'type': u'Software', u'homepage': u'https:/...",https://annotations.libcrowds.com/annotations/...,describing,https://api.bl.uk/metadata/iiif/ark:/81055/vdc...,https://api.bl.uk/metadata/iiif/ark:/81055/vdc...,Annotation
5,"[{u'type': u'TextualBody', u'purpose': u'descr...",2018-05-30T18:34:35Z,NaN,2018-06-04T09:43:56Z,"[{u'type': u'Software', u'homepage': u'https:/...",https://annotations.libcrowds.com/annotations/...,describing,https://api.bl.uk/metadata/iiif/ark:/81055/vdc...,https://api.bl.uk/metadata/iiif/ark:/81055/vdc...,Annotation
7,"[{u'type': u'TextualBody', u'purpose': u'descr...",2018-05-30T18:35:04Z,NaN,2018-06-04T09:43:56Z,"[{u'type': u'Software', u'homepage': u'https:/...",https://annotations.libcrowds.com/annotations/...,describing,https://api.bl.uk/metadata/iiif/ark:/81055/vdc...,https://api.bl.uk/metadata/iiif/ark:/81055/vdc...,Annotation
8,"[{u'type': u'TextualBody', u'purpose': u'descr...",2018-05-30T18:35:35Z,NaN,2018-06-04T09:43:56Z,"[{u'type': u'Software', u'homepage': u'https:/...",https://annotations.libcrowds.com/annotations/...,describing,https://api.bl.uk/metadata/iiif/ark:/81055/vdc...,https://api.bl.uk/metadata/iiif/ark:/81055/vdc...,Annotation
9,"[{u'type': u'TextualBody', u'purpose': u'descr...",2018-05-30T18:36:12Z,NaN,2018-06-04T09:43:56Z,"[{u'type': u'Software', u'homepage': u'https:/...",https://annotations.libcrowds.com/annotations/...,describing,https://api.bl.uk/metadata/iiif/ark:/81055/vdc...,https://api.bl.uk/metadata/iiif/ark:/81055/vdc...,Annotation


As we can see above, we now have a dataframe that contains only our transcription data. However, many of the columns, such as the body, still contain nested JSON values. We will need to process these columns further to extract the data we require. This is where reference to the [LibCrowds Data Model](https://docs.libcrowds.com/data/model/) becomes necessary in order to understand how these nested fields are structured. As the example annotation towards the top of this page shows, our describing annotations contain two bodies. One body contains a tag that indicates the entity being transcribed (e.g. the title) while the other contains the transcription as plain text. In our dataframe these two bodies are now stored as a list within the **body** column.

As well as a value, each body also contains a purpose. Similar to the motiviation of the annotation, the purpose specifies the reason that the body was created. In this case, our bodies contain two purposes: tagging and describing. The tagging body defines the entity being transcribed (e.g. title) and the describing body contains the transcription.

It would be useful to extract the values of these two bodies and add them to two additional columns in the dataframe, called **tag** and **transcription**. To do this we will write two functions that each check a list of bodies and return the value from the one with the purpose we require.

Given a list of bodies, the function below will return the value from the body with the *tagging* purpose.

In [9]:
def get_tag(bodies):
    for body in bodies:
        if body['purpose'] == 'tagging':
            return body['value']

We can then apply this function to every item in the body column of our dataframe and output the results to a new column, called **tag**.

In [10]:
df['tag'] = df['body'].apply(get_tag)

We will write a similar function to extract all of the transcriptions.

In [11]:
def get_transcription(bodies):
    for body in bodies:
        if body['purpose'] == 'describing':
            return body['value']

Then, apply it to every item in the body column and output the results to a new column, called **transcription**.

In [12]:
df['transcription'] = df['body'].apply(get_transcription)

There are remaining columns in our dataframe that still contains JSON data, such as the **target**. However, as we go on to use this dataframe in other notebooks we will see that we have a purpose for keeping the target as it is. So, for now, our dataframe manipulation is complete. 

The first few rows of our final dataset are presented below.

In [13]:
df.head()

,body,created,creator,generated,generator,id,motivation,partOf,target,type,tag,transcription
0,"[{u'type': u'TextualBody', u'purpose': u'descr...",2018-05-30T18:29:57Z,NaN,2018-06-04T09:43:56Z,"[{u'type': u'Software', u'homepage': u'https:/...",https://annotations.libcrowds.com/annotations/...,describing,https://api.bl.uk/metadata/iiif/ark:/81055/vdc...,https://api.bl.uk/metadata/iiif/ark:/81055/vdc...,Annotation,genre,Melo-Drama
5,"[{u'type': u'TextualBody', u'purpose': u'descr...",2018-05-30T18:34:35Z,NaN,2018-06-04T09:43:56Z,"[{u'type': u'Software', u'homepage': u'https:/...",https://annotations.libcrowds.com/annotations/...,describing,https://api.bl.uk/metadata/iiif/ark:/81055/vdc...,https://api.bl.uk/metadata/iiif/ark:/81055/vdc...,Annotation,genre,Asiatic Melo-Dramatic Romance
7,"[{u'type': u'TextualBody', u'purpose': u'descr...",2018-05-30T18:35:04Z,NaN,2018-06-04T09:43:56Z,"[{u'type': u'Software', u'homepage': u'https:/...",https://annotations.libcrowds.com/annotations/...,describing,https://api.bl.uk/metadata/iiif/ark:/81055/vdc...,https://api.bl.uk/metadata/iiif/ark:/81055/vdc...,Annotation,genre,Broad Comic Extravaganza
8,"[{u'type': u'TextualBody', u'purpose': u'descr...",2018-05-30T18:35:35Z,NaN,2018-06-04T09:43:56Z,"[{u'type': u'Software', u'homepage': u'https:/...",https://annotations.libcrowds.com/annotations/...,describing,https://api.bl.uk/metadata/iiif/ark:/81055/vdc...,https://api.bl.uk/metadata/iiif/ark:/81055/vdc...,Annotation,genre,Melo-Dramatic Pantomime
9,"[{u'type': u'TextualBody', u'purpose': u'descr...",2018-05-30T18:36:12Z,NaN,2018-06-04T09:43:56Z,"[{u'type': u'Software', u'homepage': u'https:/...",https://annotations.libcrowds.com/annotations/...,describing,https://api.bl.uk/metadata/iiif/ark:/81055/vdc...,https://api.bl.uk/metadata/iiif/ark:/81055/vdc...,Annotation,genre,Asiatic Melo-Dramatic Romance


We will perform one final operation to save the dataframe to disk so that it can be used in future notebooks.

In [14]:
df.to_json('../data/transcriptions.gz', compression='gzip')

## A first glance at the data

Before wrapping up, it might be interesting to introduce a few more pandas functions. 

The [describe](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.describe.html) function generates descriptive statistics that summarize the central tendency, dispersion and shape of a dataset’s distribution, excluding NaN values. We can try this out by first selecting the rows with the *genre* **tag** and adding them to a new dataframe.

In [15]:
genre_df = df[df['tag'] == 'genre']

We can then describe the **transcription** column using the following line of code.

In [16]:
genre_df['transcription'].describe()

count       2984
unique       235
top       Comedy
freq         454
Name: transcription, dtype: object

This simple function already presents some interesting results. At the time of writing, we can see that we have over 200 unique genres, with comedies being the most frequently performed. Similar summaries for *title* or *date* could be produced by entering either in place of *genre*, above.

We might already be curious about what some of the more unusual genres are. To find them, we can use the [value_counts](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.value_counts.html) function. This function returns an object containing counts of unique values. 

Below, we call this function with the argument `ascending=True`, to sort the output in ascending order, then display the first ten rows.

In [17]:
counts = genre_df['transcription'].value_counts(ascending=True)
counts[:10]

Dramatic Entertainment                   1
Operatic Farce                           1
Masquerade                               1
Grand Chinese Melo-Dramatic Spectacle    1
Pantomimical Drama                       1
Eastern Spectacles                       1
Melo Dramatic Pantomime                  1
Pantomimical Farce                       1
Serious Play                             1
Monopologue                              1
Name: transcription, dtype: int64

In a future notebook we will find out how to visualise such outputs.

## Summary

In this notebook, we found out how to load all of our results from the *In the Spotlight* crowdsourcing projects into a pandas dataframe. We then manipulated this dataframe to make the key data, such as the transcriptions, more easily accessible. We finished with a basic analysis of the genres discovered so far.

The dataframe now contains one row per transcription, many of which will contain data about the same plays. To really begin to see the benefits of the programmatic analysis methods introduced here we need some way to link these transcriptions and view all information about a play in a single row. We can then begin to pull in additional data from other sources, such as the theatre names where the plays were performed, which are stored in the [IIIF](http://iiif.io/) manifests from which the original crowdsourcing tasks were built. With this additional information we can start to answer questions such as:

- Are there any trends be found for the genres of plays performed at particular periods or across different regions?
- How likely were plays to travel between regions?
- Were some periods more innovative than others, with a greater variety of plays performed?

In the notebook [*Building A Dataframe of Plays*](intro_to_visualising_libcrowds_data_using_python.ipynb) we will see how to build on the work started here to create another dataframe where each row contains the title, genre and date for a single play.

Alternatively, for a basic introduction to producing visualisations using Python you might like to try the notebook [*An Introduction to Visualising LibCrowds Data Using Python*](intro_to_visualising_libcrowds_data_using_python.ipynb).